# Задача

Необходимо создать и запустить docker-compose с двумя контейнерами.

Первый контейнер должен быть запущен с образом `adkosmos/vault:latest` и называться `vault`. Внутри этого контейнера запускается веб-сервер который работает на порту `9090`. При запросе он отдает строку, в которой зашифрован текст.

Второй контейнер должен быть запущен с образом `adkosmos/decryptor-http:latest` и называться `server`. Внутри этого конейтнера также запускается веб-сервер на порту `9090`. При запросе он делает свой запрос на адрес `http://vault:9090/`, получает данные и возвращает декодированную информацию. Если у него не получилось сделать запрос или декодировать полученную информацию он напишет ошибку.

Ваша задача - написать docker-compose с этими контейнерами, сделать запрос к `adkosmos/decryptor-http:latest` и получить декодированную строку. Эту декодированную строку необходимо записать в файл `result.txt`

## Решение

**docker-compose.yaml**
```yaml
version: "3.7"

services:
  vault:
    image: adkosmos/vault:latest
  server:
    image: adkosmos/decryptor-http:latest
  proxy:
    image: nginx:1.17
    volumes:
      - ./configuration.nginx:/etc/nginx/nginx.conf:ro
    ports:
      - 8888:8888
```

**configuration.nginx**
```
user root;
worker_processes  4;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;

events {
    worker_connections  4096;
}

http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;
    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile            on;
    tcp_nopush          on;
    keepalive_timeout   65;

    server {
        listen  8888;

        location /vault {
            proxy_pass http://vault:9090/;   # Перенаправляем в соседний контейнер по именно контейнера!
            rewrite ^/vault/(.*) /$1 break;  # Удаляем префикс /web-server-1
        }

        location /server {
            proxy_pass http://server:9090/;
            rewrite ^/server/(.*) /$1 break;
        }
    }
}
```

In [1]:
%%writefile result.txt
I need some sleep, you can't go on like this

Writing result.txt


In [2]:
!cat result.txt

I need some sleep, you can't go on like this